In [ ]:
# Install required libraries
!apt-get install -y wkhtmltopdf
!pip install -q trafilatura pdfkit pdfplumber textblob rank_bm25 nltk spacy

# Download necessary NLTK and spaCy resources
!python -m spacy download en_core_web_sm
!python -m nltk.downloader punkt stopwords wordnet

import requests
import logging
import trafilatura
from bs4 import BeautifulSoup

# NLP and text processing libraries
import nltk
import spacy
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from rank_bm25 import BM25Okapi

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Initialize NLP resources
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nlp = spacy.load('en_core_web_sm')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define constants
COINS = ["dogecoin", "bitcoin", "ethereum", "solana", "hamster"]
BASE_URL = "https://thenewscrypto.com/page/{}/?s={}"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def preprocess_text(text):
    """Text preprocessing for better BM25 ranking."""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    return [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]

def calculate_sentiment(text):
    """Calculate sentiment and return a formatted response."""
    polarity = TextBlob(text).sentiment.polarity
    label = "Neutral" if polarity == 0 else "Positive" if polarity > 0 else "Negative"
    return f"Query Sentiment: {label} (Score: {round((polarity + 1) * 50, 1)}/100)"

def scrape_articles(coin):
    """Scrape articles for a given coin."""
    url = BASE_URL.format(1, coin)
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        articles = [a["href"] for a in soup.select("h3.card-title.fs-17 a")[:10]]
        return articles
    except Exception as e:
        logging.error(f"Error scraping {coin} articles: {e}")
        return []

def extract_content(urls):
    """Extract article content."""
    return [trafilatura.extract(trafilatura.fetch_url(url)) for url in urls if url]

def create_bm25_index(articles):
    """Create BM25 index."""
    tokenized_corpus = [preprocess_text(article) for article in articles if article]
    return {'index': BM25Okapi(tokenized_corpus), 'articles': articles, 'tokens': tokenized_corpus}

def search_articles(query, bm25_data):
    """Perform BM25 search and return top 3 results."""
    processed_query = preprocess_text(query)
    bm25, articles = bm25_data['index'], bm25_data['articles']
    scores = bm25.get_scores(processed_query)
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:3]
    return [(i + 1, articles[idx][:100] + "...", round(scores[idx], 2)) for i, idx in enumerate(top_indices)]

def main():
    """Main interactive search loop."""
    print("Welcome to the Crypto News Search!")
    print(f"Available coins: {', '.join(COINS)}")
    coin_data = {coin: create_bm25_index(extract_content(scrape_articles(coin))) for coin in COINS}

    while True:
        coin = input("Enter the coin name or type stop to exit: ").strip().lower()
        if coin == "stop": break
        if coin not in COINS:
            print("Invalid coin. Try again.")
            continue

        query = input("Enter your query: ")
        print("\n" + calculate_sentiment(query) + "\n")

        results = search_articles(query, coin_data[coin])
        print("\nTop 3 Ranked Answers:")
        for rank, snippet, score in results:
            print(f"    Rank {rank}: {snippet} (Score: {score})")

if __name__ == "__main__":
    main()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon geoclue-2.0 glib-networking glib-networking-common glib-networking-services
  gsettings-desktop-schemas iio-sensor-proxy libavahi-core7 libavahi-glib1 libdaemon0 libevdev2
  libfontenc1 libgudev-1.0-0 libhyphen0 libinput-bin libinput10 libjson-glib-1.0-0
  libjson-glib-1.0-common libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1
  libnl-genl-3-200 libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5 libqt5printsupport5 libqt5qml5
  libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5 libqt5webchannel5 libqt5webkit5
  libqt5widgets5 libsoup2.4-1 libsoup2.4-common libwacom-bin libwacom-common libwacom9 libwoff1
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0
  libxcb-xinput0 libxcb-xkb1 l